In [193]:
import numpy as np
import pandas as pd
import torch
from torch import nn

In [194]:
df = pd.read_csv("train.csv")
mean_age = df['Age'].mean()
df['Age'].fillna(value=mean_age, inplace=True)
df['Sex'] = df['Sex'].apply(lambda x: 0 if x == 'male' else 1)
# df

In [195]:
y = df['Survived']
X = df.drop(columns='Survived')
X.shape[1]

numpy_array_X = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']].values
X = torch.from_numpy(numpy_array_X).type(torch.float)
y = torch.from_numpy(y.values).type(torch.float)

In [196]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train , y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [197]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Current device: ", device)

Current device:  cpu


In [198]:
class TitanicModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer1 = nn.Linear(in_features=X.shape[1], out_features=64)
    self.layer2 = nn.Linear(in_features=64, out_features=128)
    self.layer3 = nn.Linear(in_features=128, out_features=16)
    self.layer4 = nn.Linear(in_features=16, out_features=1)
    self.relu = nn.ReLU()
  def forward(self, x):
    return self.layer4(self.relu(self.layer3(self.relu(self.layer2(self.relu(self.layer1(x)))))))

model_0 = TitanicModel().to(device)

In [199]:
len(model_0(X_train))

712

In [200]:
loss_fn = nn.BCEWithLogitsLoss()

optimizer = torch.optim.SGD(params=model_0.parameters(), lr=0.01)

In [201]:
# Training Loop

torch.manual_seed(41)
epochs = 10000

X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)

for epoch in range(epochs):
  model_0.train()

  # Forward pass
  y_logits = model_0(X_train).squeeze()
  y_pred = torch.round(torch.sigmoid(y_logits))

  # Calc loss
  loss = loss_fn(y_logits, y_train)

  # optimizer zero grad
  optimizer.zero_grad()

  # backprop
  loss.backward()

  # optimizer step gradient descent
  optimizer.step()

  # Testing
  model_0.eval()
  with torch.inference_mode():
    # Forward pass
    test_logits = model_0(X_test).squeeze()
    test_pred = torch.round(torch.sigmoid(test_logits))

    # Loss
    test_loss = loss_fn(test_logits, y_test)

    if epoch % 1000 == 0:
      print(f"Epoch: {epoch} | Loss: {loss:.4f} | Test Loss: {test_loss:.4f}")

Epoch: 0 | Loss: 0.8268 | Test Loss: 0.6334
Epoch: 1000 | Loss: 0.5784 | Test Loss: 0.5454
Epoch: 2000 | Loss: 0.5476 | Test Loss: 0.5285
Epoch: 3000 | Loss: 0.5187 | Test Loss: 0.5065
Epoch: 4000 | Loss: 0.5000 | Test Loss: 0.4968
Epoch: 5000 | Loss: 0.4829 | Test Loss: 0.4886
Epoch: 6000 | Loss: 0.4483 | Test Loss: 0.4620
Epoch: 7000 | Loss: 0.4461 | Test Loss: 0.4649
Epoch: 8000 | Loss: 0.4376 | Test Loss: 0.4627
Epoch: 9000 | Loss: 0.4217 | Test Loss: 0.4463
